# Image classification with Fastai
In this notebook you will learn how to make in a few lines of code a model that distinguises cats from dogs in pictures, using the [fastai](https://docs.fast.ai/) library.

<font color='green'>To train a deep learning model, a GPU runtime is needed. With datalayer, you can switch your kernel to a remote kernel with GPU using the kernel picker.</font>

## Disclaimer
This is widely based on the [fastbook](https://github.com/fastai/fastbook) created by [Jeremy Howard](https://github.com/jph00) and [Sylvain Grugger](https://github.com/sgugger). 


## Dataset

A dataset called the [Oxford-IIIT Pet Dataset](http://www.robots.ox.ac.uk/~vgg/data/pets/) that contains 7,349 images of cats and dogs from 37 different breeds will be downloaded from the fast.ai datasets collection and will then be extracted.

In [ ]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)/'images'  # downloads the data

def is_cat(x): return x[0].isupper()  # label function (if uppercase it is a cat, otherwise it is a dog)

dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))  # data loader

We can then check if everything looks okay with the `show_batch` method (`True` is for cat, `False` is for dog):

In [ ]:
dls.show_batch()

## Model training

A *pretrained model* that has already been trained on 1.3 million images, using a competition-winning model will be downloaded from the internet.

The pretrained model will be *fine-tuned* using the latest advances in transfer learning, to create a model that is specially customized for recognizing dogs and cats. It will run two epochs, one only on 

The fine-tuning needs to be run on GPU.

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate, cbs=ShowGraphCallback())  # our CNN model
learn.fine_tune(2)  # training of the model

## Predict on new data

In [ ]:
! curl https://raw.githubusercontent.com/fastai/fastbook/master/images/chapter1_cat_example.jpg -o /tmp/chapter1_cat_example.jpg
! ls /tmp/chapter1_cat_example.jpg

In [ ]:
pred, _, probs = learn.predict("/tmp/chapter1_cat_example.jpg")
print(f"Is this a cat?: {pred}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")

## Saving our model

In [ ]:
learn.export('export.pkl')